In [1]:
import json
import ast
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
import os
import tqdm

[NeMo W 2021-09-09 16:24:03 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-09-09 16:24:07 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
nemo_asr.models.EncDecCTCModelBPE.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=stt_en_citrinet_256,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_citrinet_256,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_256/versions/1.0.0rc1/files/stt_en_citrinet_256.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=stt_en_citrinet_512,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_citrinet_512,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_512/versions/1.0.0rc1/files/stt_en_citrinet_512.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=stt_en_citrinet_1024,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_citrinet_1024,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_1024/versions/1.0.0rc1/files/stt_en_citrinet_1024.nemo
 ),
 PretrainedM

In [3]:
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2021-09-09 16:26:37 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2021-09-09 16:27:15 common:676] Instantiating model from pre-trained checkpoint
[NeMo I 2021-09-09 16:27:16 features:252] PADDING: 16
[NeMo I 2021-09-09 16:27:16 features:269] STFT using torch
[NeMo I 2021-09-09 16:27:16 modelPT:438] Model EncDecCTCModel was successfully restored from /Users/dami.osoba/.cache/torch/NeMo/NeMo_1.2.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [3]:
asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small_ls")

[NeMo I 2021-09-06 09:16:06 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small_ls/versions/1.0.0/files/stt_en_conformer_ctc_small_ls.nemo to /home/ubuntu/.cache/torch/NeMo/NeMo_1.3.0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo
[NeMo I 2021-09-06 09:16:07 common:681] Instantiating model from pre-trained checkpoint
[NeMo I 2021-09-06 09:16:08 mixins:147] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2021-09-06 09:16:08 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/Librispeech_SP_Tarred/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/Librispeech_SP_Tarred/audio__OP_0..511_CL_.tar
    
[NeMo W 2021-09-06 09:16:08 modelPT:137] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manife

[NeMo I 2021-09-06 09:16:08 features:252] PADDING: 0
[NeMo I 2021-09-06 09:16:08 features:269] STFT using torch
[NeMo I 2021-09-06 09:16:08 save_restore_connector:143] Model EncDecCTCModelBPE was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.3.0/stt_en_conformer_ctc_small_ls/cf1b6bbcc08433257c12442c92b9996a/stt_en_conformer_ctc_small_ls.nemo.


In [4]:
# download s3://model-data-platform/validated/  for manifest and wav files
def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm.tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest
# train_manifest_path = '/Users/dami.osoba/work/bawk/src/data/commonvoice_train_manifest.json'
train_manifest_path = "../../commonvoice_validated_manifest.json"
train_manifest_data = read_manifest(train_manifest_path)
train_path = [(data['audio_filepath'].replace('validated','wav_clips'), data['text']) for data in train_manifest_data]

FileNotFoundError: [Errno 2] No such file or directory: '../../commonvoice_validated_manifest.json'

In [20]:
files = [a[0] for a in train_path[:100]]
transcription = [a[1] for a in train_path[:100]]

In [22]:
# get transcriptions 
hypothesis = asr_model.transcribe(paths2audio_files=files[:100])


Transcribing:   0%|          | 0/25 [00:00<?, ?it/s]

In [33]:
hypothesis[:5]

['six o',
 'all is well that ends well',
 'loyal in me',
 'the new patch is less invasing than the ole one but still causes redrection',
 'how is monsieur a going to bandle and be with is a due and cue']

In [34]:
#actual
transcription[:5]

['six',
 "all's well that ends well.",
 'do you mean it?',
 'the new patch is less invasive than the old one, but still causes regressions.',
 'how is mozilla going to handle ambiguities like queue and cue?']

In [25]:
#get word error rate
wer = word_error_rate(hypotheses=hypothesis, references=transcription)
print(wer)

0.5379388448471121
